In [1]:
import torch
from modelscope import Qwen3VLForConditionalGeneration, AutoProcessor
from PIL import Image

# Set device and load model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = Qwen3VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen3-VL-4B-Instruct", 
    dtype=torch.float16,
    device_map="auto"
).to(device)

processor = AutoProcessor.from_pretrained("Qwen/Qwen3-VL-4B-Instruct")

d:\files\envs\qwen\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.55s/it]


In [76]:
# Load and prepare image
image_path = r"D:\Tsingtao\media\image263.png"
image = Image.open(image_path).convert('RGB')



# Prepare message with local image (最终优化版，强调占道经营的绝对性)
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": image  # Direct image object instead of URL
            },
            {
                "type": "text", 
                "text": "你是一名正在巡逻的城市管理执法人员（城管）。你的任务是：1. **城管重点巡查目标**：立即识别并报告任何**占道经营**、乱堆乱放、违规广告、违规停车、路面破损、流动商贩等问题。2. **占道经营判定**：**任何**将商品（包括但不限于服装、展示架）放置在房屋的**墙体或门槛线外**、占用公共区域的行为，**属于违规占道经营 (如果在店内则无问题）**。店内可见的货物不视为占道。3. **排除误判**：正常生长的行道树，未构成实质性通行障碍则不视为问题。4. 若发现违规，**仅用一句话**描述核心问题；若未发现，则**无需输出任何内容**。直接执行。"
            }
        ]
    }
]


# Preparation for inference
inputs = processor.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_dict=True,
    return_tensors="pt"
)
inputs = inputs.to(model.device)

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=512)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

['未发现违规占道经营或其他城市管理问题。']


In [ ]:
# messages = [
#     {
#         "role": "user",
#         "content": [
#             {
#                 "type": "image",
#                 "image": image  # Direct image object instead of URL
#             },
#             {"type": "text", "text": "你做为一名正在日常巡检的城管，这里有什么城市管理方面的问题吗? 用一到两句话描述"}
#         ]
#     }
# ]

# messages = [
#     {
#         "role": "user",
#         "content": [
#             {
#                 "type": "image",
#                 "image": image  # Direct image object instead of URL
#             },
#             {"type": "text", "text": "你是城市管理工作的智能助手，需从图片中识别以下城市管理违规问题，输出结果需包含 “违规类型 + 关键场景描述，只用两句话输出有问题的地方"
#             "无照游商：流动商贩未取得经营许可，在道路、通道口、路口等公共区域摆摊（如售卖海鸥食、水果、小商品等）。"
#             "店外经营：商户超出店铺规划范围，在门口、南侧等区域摆放商品经营（如水果蔬菜、茶叶、鞋垫子、啤酒桶、桌椅马扎等）。"
#             "设施问题：垃圾桶未归位、人行道便道桩损坏脱落、非清运时间将垃圾箱摆放在车行道。"}
#         ]
#     }
# ]

# # Prepare message with local image
# messages = [
#     {
#         "role": "user",
#         "content": [
#             {
#                 "type": "image",
#                 "image": image  # Direct image object instead of URL
#             },
#             {"type": "text", "text": "你是一名正在巡逻的城管。请用一句话，直接指出本场景中你最关注的、需要立即处理的城市管理突出问题 （例如：占道经营、乱堆乱放、 路面破损等)。只输出问题，少说废话。"}
#         ]
#     }
# ]

# # Prepare message with local image (使用最终优化、包含静默和排错逻辑的提示词)
# messages = [
#     {
#         "role": "user",
#         "content": [
#             {
#                 "type": "image",
#                 "image": image  # Direct image object instead of URL
#             },
#             {
#                 "type": "text", 
#                 "text": "你是一名正在巡逻的城市管理执法人员（城管）。你的任务是：1. 仅识别的城管职责范围内的违规行为（例如：占道经营、乱堆乱放、 路面破损等) 和其余非常紧急的各种问题。2. **忽略**所有轻微或符合规范的正常现象（例如：**正常生长的行道树，只要未构成实质性通行障碍，则不视为问题**）。3. 若存在严重问题，**仅用一句话**描述；若不存在，则**无需输出任何内容**。"
#             }
#         ]
#     }
# ]

# Prepare message with local image (使用最终优化、包含静默和排错逻辑的提示词)
# messages = [
#     {
#         "role": "user",
#         "content": [
#             {
#                 "type": "image",
#                 "image": image  # Direct image object instead of URL
#             },
#             {
#                 "type": "text", 
#                 "text": "你是一名正在巡逻的城市管理执法人员（城管）。你的任务是：1. 仅识别**突出、急需立即干预**的城管职责范围内的违规行为（例如：占道经营、乱堆乱放、违规停车、路面破损、流动商贩等)。2. **占道经营判断**：必须确认货物或经营活动已**实际跨越**房屋墙体或门槛，**占用公共区域**（如人行道）。店内可见的货物不视为占道。3. **排除误判**：正常生长的行道树，只要未构成实质性通行障碍，不视为问题。4. 若存在严重问题，**仅用一句话**描述；若不存在，则**无需输出任何内容**。请直接执行。"
#             }
#         ]
#     }
# ]